In [14]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.utils import image_dataset_from_directory
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from google.colab import drive
from google.colab import files # Import files module here
import zipfile
import os
from PIL import Image
import io

print("All libraries imported successfully.")

# Mount Google Drive
drive.mount('/content/drive')

All libraries imported successfully.
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [15]:
# Load the standard MNIST dataset from your Drive
mnist_csv_path = '/content/drive/MyDrive/MNIST_CPP_Project/train.csv'

print(f"Loading standard MNIST data from {mnist_csv_path}...")
df = pd.read_csv(mnist_csv_path)

X = df.iloc[:, 1:].values
y = df.iloc[:, 0].values

# Normalize pixel values to be between 0 and 1
X = X / 255.0

# Reshape data for the CNN model (num_samples, height, width, channels)
X_reshaped = X.reshape(-1, 28, 28, 1)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_reshaped, y, test_size=0.2, random_state=42)

print("\nStandard MNIST data prepared.")
print(f"Training data shape: {X_train.shape}")
print(f"Validation data shape: {X_val.shape}")

Loading standard MNIST data from /content/drive/MyDrive/MNIST_CPP_Project/train.csv...

Standard MNIST data prepared.
Training data shape: (33600, 28, 28, 1)
Validation data shape: (8400, 28, 28, 1)


In [16]:
!ls /content/My_Custom_Digits/

0  1  2  3  4  5  6  7	8  9


In [17]:
# Path to your uploaded zip file
zip_file_path = '/content/drive/MyDrive/MNIST_CPP_Project/Final_Custom_Dataset.zip'
# Path to extract the files to
extract_path = '/content/My_Custom_Digits/'

print(f"Extracting {zip_file_path}...")
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)
print("Extraction complete!")

Extracting /content/drive/MyDrive/MNIST_CPP_Project/Final_Custom_Dataset.zip...
Extraction complete!


In [18]:
# Path to the directory containing the subfolders (0, 1, 2...)
# We are removing 'Final_Custom_Dataset' from the path
custom_dataset_dir = '/content/My_Custom_Digits/'

# Load your custom dataset using Keras utility
my_custom_dataset = image_dataset_from_directory(
    custom_dataset_dir,
    labels='inferred',
    label_mode='int',
    color_mode='grayscale',
    image_size=(28, 28),
    batch_size=32,
    shuffle=True
)

# Convert the dataset into NumPy arrays for processing
my_images_list = []
my_labels_list = []
for images, labels in my_custom_dataset:
    my_images_list.append(images.numpy())
    my_labels_list.append(labels.numpy())

my_images = np.concatenate(my_images_list, axis=0)
my_labels = np.concatenate(my_labels_list, axis=0)

# Normalize your custom images
my_images_normalized = my_images / 255.0

print("\nYour custom dataset has been loaded and prepared.")
print(f"Shape of your image data: {my_images_normalized.shape}")

Found 21555 files belonging to 10 classes.

Your custom dataset has been loaded and prepared.
Shape of your image data: (21555, 28, 28, 1)


In [19]:
# Combine the standard training data with your custom data
# The variables X_train and y_train should be available from Step 2
X_combined_train = np.concatenate((X_train, my_images_normalized))
# The variable name was 'my_labels' in the previous step, not 'y_labels'
y_combined_train = np.concatenate((y_train, my_labels))

print("Standard and custom datasets have been combined.")
print(f"Old training data count: {len(X_train)}")
print(f"Your custom data count: {len(my_images_normalized)}")
print(f"New combined training data count: {len(X_combined_train)}")
print(f"New combined training data shape: {X_combined_train.shape}")

Standard and custom datasets have been combined.
Old training data count: 33600
Your custom data count: 21555
New combined training data count: 55155
New combined training data shape: (55155, 28, 28, 1)


In [20]:
# Create the CNN model
model = keras.Sequential([
    keras.Input(shape=(28, 28, 1)),
    layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Flatten(),
    layers.Dropout(0.5),
    layers.Dense(10, activation="softmax"),
])

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Print a summary of the model architecture
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 26, 26, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 13, 13, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 11, 11, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 5, 5, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 1600)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1600)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 10)             │        16,010 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 34,826 (136.04 KB)

 Trainable params: 34,826 (136.04 KB)

 Non-trainable params: 0 (0.00 B)

In [21]:
print("Starting the final model training with the combined dataset...")
print(f"Training on {len(X_combined_train)} images.")

history = model.fit(
    X_combined_train,
    y_combined_train,
    epochs=15,  # We can train for more epochs as we have a larger dataset
    validation_data=(X_val, y_val),
    batch_size=32
)

print("\nModel training is complete!")

Starting the final model training with the combined dataset...
Training on 55155 images.
Epoch 1/15
1724/1724 ━━━━━━━━━━━━━━━━━━━━ 51s 29ms/step - accuracy: 0.6995 - loss: 0.9037 - val_accuracy: 0.9740 - val_loss: 0.0902
Epoch 2/15
1724/1724 ━━━━━━━━━━━━━━━━━━━━ 47s 27ms/step - accuracy: 0.9122 - loss: 0.2872 - val_accuracy: 0.9811 - val_loss: 0.0639
Epoch 3/15
1724/1724 ━━━━━━━━━━━━━━━━━━━━ 82s 27ms/step - accuracy: 0.9348 - loss: 0.2188 - val_accuracy: 0.9831 - val_loss: 0.0562
Epoch 4/15
1724/1724 ━━━━━━━━━━━━━━━━━━━━ 86s 30ms/step - accuracy: 0.9429 - loss: 0.1924 - val_accuracy: 0.9838 - val_loss: 0.0553
Epoch 5/15
1724/1724 ━━━━━━━━━━━━━━━━━━━━ 80s 28ms/step - accuracy: 0.9513 - loss: 0.1610 - val_accuracy: 0.9858 - val_loss: 0.0455
Epoch 6/15
1724/1724 ━━━━━━━━━━━━━━━━━━━━ 83s 29ms/step - accuracy: 0.9559 - loss: 0.1455 - val_accuracy: 0.9877 - val_loss: 0.0399
Epoch 7/15
1724/1724 ━━━━━━━━━━━━━━━━━━━━ 84s 30ms/step - accuracy: 0.9563 - loss: 0.1373 - val_accuracy: 0.9892 - val_

In [ ]:
print("Upload a new handwritten digit image to test the final model.")

# This command opens a file upload dialog in the cell's output
uploaded = files.upload()

if not uploaded:
    print("\nNo file was selected.")
else:
    # Get the filename and content of the first uploaded file
    filename = next(iter(uploaded))
    content = uploaded[filename]

    print(f"\nProcessing the uploaded file: {filename}...")

    try:
        # Open the uploaded image from its byte content
        img = Image.open(io.BytesIO(content)).convert('L')

        # Resize the image to 28x28 pixels
        img = img.resize((28, 28), Image.Resampling.LANCZOS)

        # Convert the image to a NumPy array of pixel values
        pixel_data = np.array(img)

        # Heuristic to detect if the background is white (common for user drawings)
        # If the mean pixel value of the corners is high, it's likely a white background
        if np.mean(pixel_data[0:5, 0:5]) > 200 and np.mean(pixel_data[-5:, -5:]) > 200:
            print("White background detected. Inverting colors for the model.")
            pixel_data = 255 - pixel_data

        # Normalize the pixel values to be between 0 and 1
        pixel_data = pixel_data / 255.0

        # Reshape the data to the format the model expects: (1, 28, 28, 1)
        processed_image = pixel_data.reshape(1, 28, 28, 1)

        # Use the trained model to make a prediction
        predictions = model.predict(processed_image)
        predicted_digit = np.argmax(predictions)

        print("\n=====================================")
        print(f"The model predicts the digit is: {predicted_digit}")
        print("=====================================")

    except Exception as e:
        print(f"\nAn error occurred while processing the image: {e}")
        print("Please make sure you have uploaded a valid image file (e.g., PNG, JPG).")

Upload a new handwritten digit image to test the final model.
